In [ ]:
%load_ext autoreload
%autoreload 2

import os
import uproot
import matplotlib as mpl

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm, linregress
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.optimize import curve_fit
from scipy import stats
import datetime as dt
import scipy.optimize
import landau
from scipy.interpolate import CubicSpline, interp2d, interp1d
from scipy.integrate import quad
from scipy.special import erf
from numpy import linalg

import importlib
from multiprocessing import Pool

from tqdm.auto import tqdm
# local imports
from lib.constants import *

from matplotlib import collections  as mc

In [ ]:
1+1

In [ ]:
import emcee
import corner

In [ ]:
dosave = False
plt.rcParams.update({'font.size': 14})
plotqual = ""
savedir = "./plots_11_14_23/run1_data_blehh_"

plottitle = "Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]
# datadir = "/icarus/data/users/gputnam/DMCP2023G/calib-data/"
datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

isMC = False
PHICORRECT = False
cut_TPCEE = False

savedata = False

In [ ]:
# Constants
if isMC:
    LAr_density_gmL = 1.389875
else:
    LAr_density_gmL = 1.3926
    
mass_electron = 0.5109989461 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
muon_mass = 105.6583745 # MeV https://pdg.lbl.gov/2020/listings/rpp2020-list-muon.pdf

proton_mass = 938.272

if not isMC:
    Ival = 197.0e-6
else:
    Ival = 188e-6
    
Zval = 18.0
Aval = 39.948
# Kfactor = 0.307075

Ar_molar_mass = 39.9623
Ar_ZA = 18. / Ar_molar_mass
Relec = 2.817940 * 1e-13
mole = 6.0221409*1e23
Kfactor = 4*np.pi*mole*Relec**2*mass_electron # 0.307075


In [ ]:
def Calc_MEAN_DEDX(T, thisIval=Ival, mass=muon_mass):
    gamma = (mass+T)/mass
    beta = np.power(1.0-np.power(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+np.power(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*np.power(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    dens_factor[beta < 1e-6] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    dEdx_mean = LAr_density_gmL*Kfactor*(Zval/Aval)*np.power(beta,-2.0)*(0.5*np.log(2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0)*Wmax*np.power(thisIval,-2.0))-np.power(beta,2.0)-dens_factor/2.0)

    return dEdx_mean


def Calc_MPV_DEDX(thick, T, thisIval=Ival, mass=muon_mass):
    gamma = (mass+T)/mass
    beta = np.power(1.0-np.power(gamma,-2.0),0.5)
    Wmax = (2.0*mass_electron*np.power(beta,2.0)*np.power(gamma,2.0))/(1.0+2.0*gamma*(mass_electron/mass)+np.power(mass_electron/mass,2.0))

    # Medium energy 
    dens_factor = 2.0*np.log(10)*np.log10(beta*gamma)-5.2146+0.19559*np.power(3.0-np.log10(beta*gamma),3.0)
    # low energy
    dens_factor[np.log10(beta*gamma) < 0.2] = 0.
    # high energy
    dens_factor[np.log10(beta*gamma) > 3.0] = (2.0*np.log(10)*np.log10(beta*gamma)-5.2146)[np.log10(beta*gamma) > 3.0]
    xi = (Kfactor/2.0)*(Zval/Aval)*np.power(beta,-2.0)*LAr_density_gmL*thick
    dEdx_MPV = xi*(np.log((2.0*mass_electron*np.power(beta*gamma,2.0))/thisIval)+np.log(xi/thisIval)+0.200-np.power(beta,2.0)-dens_factor)/thick
    return dEdx_MPV

def Calc_LOC_DEDX(thick, T, mass=muon_mass):
    gamma = (mass+T)/mass
    beta = np.power(1.0-np.power(gamma,-2.0),0.5)
    xi = (Kfactor/2.0)*(Zval/Aval)*np.power(beta,-2.0)*LAr_density_gmL*thick
    return Calc_MPV_DEDX(thick, T) - 0.22278*xi/thick


def Calc_RR_points(KE, dRR=0.01, mass=muon_mass):
    thisKE = KE
    KE_points = [thisKE]
    RR_points = [0.]

    while thisKE > 0.0:
        deltaKE = Calc_MEAN_DEDX(np.array([thisKE]), mass=mass) * dRR
        RR_points.append(RR_points[-1] + dRR)
        thisKE -= deltaKE[0]
        KE_points.append(thisKE)

    KE_points = np.array(list(reversed(KE_points[:-1])))
    RR_points = np.array(RR_points[:-1])

    return KE_points, RR_points

KE_points_max = 1000.
KE_points, RR_points = Calc_RR_points(KE_points_max, mass=muon_mass)

RR2KE_mu = CubicSpline(RR_points, KE_points)

KE_points_max = 1000.
KE_points, RR_points = Calc_RR_points(KE_points_max, mass=proton_mass)

RR2KE_p = CubicSpline(RR_points, KE_points)


In [ ]:
def RRthick2dEdxMu(RR, thick, thisI=Ival):
    return Calc_MPV_DEDX(thick, RR2KE_mu(RR), thisI, muon_mass)

def RRthick2dEdxP(RR, thick, thisI=Ival): 
    return Calc_MEAN_DEDX(RR2KE_p(RR), thisI, proton_mass)

def RRthick2dEdx(RR, thick, is_muon, thisI=Ival):
    ret = RRthick2dEdxMu(RR, thick)
    ret[~is_muon] = RRthick2dEdxP(RR, thick)[~is_muon]

    return ret

In [ ]:
if isMC:
    mpv_file = "/icarus/data/users/gputnam/KE_length_2_MPV_I188.npy" # MC value
else:
    mpv_file = "/icarus/data/users/gputnam/KE_length_2_MPV_I197.npy" # best guess data value

with open(mpv_file, "rb") as f:
    ke_arr = np.load(f)
    length_arr = np.load(f)
    mpv_arr = np.load(f)
    
ke_length_2_mpv = interp2d(ke_arr, length_arr, mpv_arr, kind="linear")

mpv_file

In [ ]:
def RR2LVMPVdEdx(RR, thick):
    return np.array([ke_length_2_mpv(RR2KE_p(rr), t)[0] for rr, t in zip(RR, thick)])

In [ ]:
np.diag(ke_length_2_mpv(RR2KE_p([5, 20]), [0.4, 1])), ke_length_2_mpv(RR2KE_p(20), 0.4)

In [ ]:
pltrrs = np.linspace(1, 200, 101)
plt.plot(pltrrs, RRthick2dEdxMu(pltrrs, 0.6))

In [ ]:
def f_zeta(RR, thick, mass=muon_mass):
    KE = RR2KE_mu(RR)
    muonE = KE + mass
    
    gamma = muonE / mass
    beta = np.sqrt(1 - (1/gamma)**2)

    gammaB2 = gamma*gamma*beta*beta
    maxE = 2 * mass_electron * gammaB2 / \
        ( 1 + 2 * gamma * mass_electron / mass + (mass_electron/mass)**2)

    zeta = (Kfactor/2.)*Ar_ZA*(1./beta**2) * LAr_density_gmL / maxE
    
    return zeta*thick

In [ ]:
rr_plot = np.linspace(20, 200, 101)
plt.plot(rr_plot, f_zeta(rr_plot, 1))
plt.xlabel("Residual Range [cm]")
plt.ylabel("$\\zeta$ [cm$^{-1}$]")
plt.axhline([0.01], color="black", linestyle="--")
plt.text(100, 0.0125, "^ Landau MPV NOT Valid\nat $\\mathscr{t} = 1$ cm")
plt.tight_layout()

if dosave: plt.savefig(savedir + "zeta_plot.pdf")
if dosave: plt.savefig(savedir + "zeta_plot.svg")

In [ ]:
thicks = 0.01 / f_zeta(rr_plot, 1)
plt.plot(thicks, rr_plot)
plt.ylabel("Lowest Landau-Valid\nResidual Range [cm]")
plt.xlabel("Channel Thickness [cm]")
plt.tight_layout()

if dosave: 
    plt.savefig(savedir + "valid_rr.pdf")
    plt.savefig(savedir + "valid_rr.svg")
    

In [ ]:
wirep = 0.3
if isMC:
    Dtransverse = 8.8e-3 # cm^2/ms
else:
    Dtransverse = 7.5e-3 # cm^2/ms
    
def smeared_dep(x0, w, sigma):
    if sigma < 1e-4:
        return 1*((x0 > -w/2) & (x0 <= w/2))
    return (1./2)*(erf((w/2+x0)/(np.sqrt(2)*sigma)) +\
                   erf((w/2-x0)/(np.sqrt(2)*sigma)))

def f_thickness(sigma, a=wirep):
    return a*np.exp(quad(lambda x: -smeared_dep(x, a, sigma) * np.log(smeared_dep(x, a, sigma))/a, 
                       -(a/2) - 5*sigma, (a/2) + 5*sigma)[0])

def smearing(driftT, Dt=Dtransverse):
    return np.sqrt(2*Dt*driftT)

smearings = np.linspace(0, max(smearing(1.25), wirep), 1000)
thickss = [f_thickness(s, 1) for s in smearings]
smear_2_thick = CubicSpline(smearings, thickss)

def thickness(pitch, time, Dt=Dtransverse):
    return (pitch) * smear_2_thick(smearing(time/1e3, Dt) / pitch)

In [ ]:
names = [
    "TPC_EE",
    "TPC_EW",
    "TPC_WE",
    "TPC_WW"
]

if isMC:
    names = ["MC"]

In [ ]:
if not isMC:
    if cut_TPCEE:
        mpv_df_p = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_proton_run1_reprodC_chi2u40_chi2p80_noTPCEE.df")
    else:
        mpv_df_p = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_proton_run1_reprodC_chi2u40_chi2p80.df")
#         mpv_df_p = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_proton_run1_reprodC_chi2u32_chi2p100.df")
        
    # mpv_df_p = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_proton_pida_physicsdata.df")
    # mpv_df_p = pd.read_hdf("/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-data/mpvs_proton_angcorr_physicsdata.df")
else:
#     mpv_df_p = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_proton_angcorr_mcfitwvf.df")
    mpv_df_p = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/EMB/mcnuphase2F-fakeEMB2-mpvs.df")
    
    
mpv_df_p["muon"] = False
mpv_df_p["tdrift"] = 500
mpv_df_p["tlo"] = 0
mpv_df_p["thi"] = 1000

mpv_df_p.mpv_err = np.sqrt((mpv_df_p.mpv*0.01)**2 + mpv_df_p.mpv_err**2) 

for n in names:
    mpv_df_p[n] = True
    
columns = mpv_df_p.columns

In [ ]:
# phi correct
if PHICORRECT:
    corr = np.array([0.97465892, 0.97440618, 0.97924557, 0.9850034 , 0.99386926,
       1.        ])
    for i, phi in enumerate(sorted(mpv_df_p.phi.unique())):
        mpv_df_p.loc[mpv_df_p.phi == phi, "mpv"] /= corr[i]
        # mpv_df_p.loc[mpv_df_p.phi == phi, "mpv_err"] /= corr[i]

In [ ]:
mpv_df_p

In [ ]:
if not isMC:
    # mpv_df_mu = pd.read_hdf("/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-data/mpvs_muon_physicsdata.df")
#     mpv_df_mu = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_muon_thxwandphicut_run1data.df")
    mpv_df_mu = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_muon_phi70-80_run1data.df")
else:
    # mpv_df_mu = pd.read_hdf("/pnfs/icarus/persistent/users/gputnam/DMCP2023G/calib-data/mpvs_muon_angcut_mcfitwvf.df")
    mpv_df_mu = pd.read_hdf("/icarus/data/users/gputnam/DMCP2023G/calib-data/mpvs_muon_thxwandphicut_mcfitwvf.df")
    
mpv_df_mu["muon"] = True

mpv_df_mu = mpv_df_mu[columns].copy()

In [ ]:
CALS = {}

if not isMC:
    with open(datadir + "tpc_ratio_Run1.txt") as f:
        for line in f:
            TPC, CAL = line.rstrip("\n").split(" ")
            CALS["TPC_" + TPC] = float(CAL)
else:
    CALS["MC"] = 1
CALS

In [ ]:
for n in names:
    mpv_df_mu.loc[mpv_df_mu[n], ["mpv", "mpv_err"]] *= CALS[n]

In [ ]:
mpv_df = pd.concat([mpv_df_mu, mpv_df_p]).reset_index()

In [ ]:
mpv_df

In [ ]:
LAr_density_gmL

In [ ]:
# ArgoNeuT params
MODA = 0.930
MODB = 0.212
Wion = 1e3 / 4.237e7
if isMC: # MC Efield
    Efield = 0.494
else: # data efield
    Efield = 0.4926
    
EFIELD_ERR = 0.017

print(Efield, Efield*EFIELD_ERR)

def recombination_box(dEdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)

    dQdx = np.log(alpha + dEdx*beta) / (Wion * beta)
    return dQdx

def recombination_cor(dQdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)
        
    dEdx = (np.exp(dQdx*Wion*beta)- alpha) / beta
        
    return dEdx

def recombination_birks(dEdx, A=0.8, B=0.0486, E=Efield):
    alpha = A
    k = B / (LAr_density_gmL * E)
    R = A / (1 + k*dEdx)
    dQdx = dEdx*R/Wion
    
    return dQdx


def recombination_elbox(dEdx, A0, eps, E=Efield):
    polylog = np.vectorize(mpmath.polylog)
    eps = eps / (LAr_density_gmL * E)
    
    # Fix the MIP dQdx to the mod box model
    dQdx0 = -(1/eps/Wion)*polylog(2, -eps*1.6)
    A = A0*recombination_box(1.6)/dQdx0
    
    dQdx = -(A/eps/Wion)*polylog(2, -eps*dEdx)
    return np.array(dQdx, dtype=float)

Wion

In [ ]:
100*(recombination_box(1.6, B=0.203) - recombination_box(1.6, B=0.206)) / recombination_box(1.6, B=0.203)

In [ ]:
dEdxs = [1.6, 1.7, 1.8, 1.9, 2, 2.1]

alpha = 0.906
betas = np.linspace(0.2, 0.3, 1001)

for dEdx in dEdxs:
    plt.plot(betas, recombination_box(dEdx, alpha, betas) / (dEdx/Wion), label="%.1f"% dEdx)
plt.legend(title="dEdx [MeV/cm]", ncol=2)

In [ ]:
# Compute the expected dE/dx

# Muons -- Landau MPV w/ thickness
mpv_df["dedx_exp"] = RRthick2dEdxMu(mpv_df.rr, thickness(mpv_df.pitch, mpv_df.tdrift))
# Proton -- LV MPV w/ pitch
mpv_df.loc[~mpv_df.muon, ["dedx_exp"]] = RR2LVMPVdEdx(mpv_df.rr[~mpv_df.muon], mpv_df.pitch[~mpv_df.muon])

# Try no diffusion
mpv_df["dedx_exp_pitch"] = RRthick2dEdxMu(mpv_df.rr, mpv_df.pitch)
mpv_df.loc[~mpv_df.muon, ["dedx_exp_pitch"]] = RR2LVMPVdEdx(mpv_df.rr[~mpv_df.muon], mpv_df.pitch[~mpv_df.muon])

In [ ]:
pitchbins = sorted(mpv_df.pitch.unique())
driftbins = sorted(mpv_df.tdrift.unique())[1:]

In [ ]:
mpv_valid = ((f_zeta(mpv_df.rr, thickness(mpv_df.pitch, mpv_df.tdrift)) < 0.01) &\
    (mpv_df.pitch == min(pitchbins)) &\
    (((not cut_TPCEE) | ~mpv_df.muon | ~mpv_df.TPC_EE) if not isMC else True) &\
    (mpv_df.tdrift >= min(driftbins)) |\
    (~mpv_df.muon & (mpv_df.rr >= 2))) & np.isfinite(mpv_df.mpv_err)

In [ ]:
mpv_df.rr[~mpv_df.muon].min()

In [ ]:
# Analysis of muon errors

In [ ]:
whenmu = mpv_valid & mpv_df.muon
plt.scatter(mpv_df[whenmu].dedx_exp, mpv_df[whenmu].mpv, label="Data")

lin = linregress(mpv_df[whenmu].dedx_exp, mpv_df[whenmu].mpv)

dedxs = np.linspace(1.63, 1.72, 100)
plt.plot(dedxs, lin.intercept + dedxs*lin.slope, color="red", label="Linear Regression")

plt.xlabel("Expected dE/dx [MeV/cm]")
plt.ylabel("Measured dQ/dx [ADC/cm]")
plt.legend()

if dosave:
    plt.savefig(savedir + "muon_data_linregress.pdf")
    plt.savefig(savedir + "muon_data_linregress.svg")

In [ ]:
exp_mpv = (lin.intercept + mpv_df.dedx_exp*lin.slope)
residuals = (mpv_df.mpv - exp_mpv) / exp_mpv
bins = 100*(np.linspace(-0.05, 0.05, 21) + 0.0025)
centers = (bins[1:] + bins[:-1]) / 2

N,_,_ = plt.hist(100*residuals[whenmu], bins=bins, label="Muon MPVs")

def gaus(x, A, mu, sigma):
    return A*np.exp(-(x-mu)**2/(2*sigma**2))

popt, perr = curve_fit(gaus, centers, N)

pltcenters = np.linspace(bins[0], bins[-1], 100)
plt.plot(pltcenters, gaus(pltcenters, *popt), label="Gaussian Fit", linewidth=2)

width = abs(popt[-1])

plt.text(0.05, 0.8, "Width: %.2f%%" % width, transform=plt.gca().transAxes)

plt.xlabel("Residual to Linear Regression [%]")
plt.ylabel("Entries")
plt.legend()

if dosave:
    plt.savefig(savedir + "muon_data_linregress_residuals.pdf")
    plt.savefig(savedir + "muon_data_linregress_residuals.svg")

In [ ]:
_ = plt.hist(100*(mpv_df.mpv_err / mpv_df.mpv)[whenmu], label="Muon MPVs")
plt.axvline(np.mean(100*(mpv_df.mpv_err / mpv_df.mpv)[whenmu]), 
            color="orange", label="Mean Muon MPV Uncertainty", linewidth=2)

plt.axvline(width, color="red", label="Bootstrapped Uncertainty", linewidth=2)
plt.xlabel("MPV Uncertainty [%]")
plt.ylabel("Entries")

plt.legend()

if dosave:
    plt.savefig(savedir + "muon_data_mpv_error.pdf")
    plt.savefig(savedir + "muon_data_mpv_error.svg")

In [ ]:
plt.scatter(100*np.abs(residuals[whenmu]), 100*(mpv_df.mpv_err / mpv_df.mpv)[whenmu], label="Muon MPVs")
plt.xlabel("Residual to Linear Regression [%]")
plt.ylabel("MPV Uncertainty [%]")
plt.legend()

if dosave:
    plt.savefig(savedir + "muon_mpv_uncertainty_residual.pdf")
    plt.savefig(savedir + "muon_mpv_uncertainty_residual.svg")

In [ ]:
phis = mpv_df[~mpv_df.muon].phi.unique()
philos = mpv_df[~mpv_df.muon].philo.unique()
phihis = mpv_df[~mpv_df.muon].phihi.unique()

nphi = len(phis)

In [ ]:
# FIT GAIN ONLY, muons only

In [ ]:
def fit_gain(dedx, gain):
    return recombination_box(dedx) / gain

In [ ]:

p0 = [
    80,
]

plo = [
    60,
]
phi = [
    90,
] 


popt, perr = curve_fit(fit_gain, 
                       mpv_df.dedx_exp[mpv_valid & mpv_df.muon], 
                       mpv_df[mpv_valid & mpv_df.muon].mpv, 
                       sigma=mpv_df[mpv_valid & mpv_df.muon].mpv_err,
                       p0=p0, bounds=(plo, phi), absolute_sigma=True)

err = np.sqrt(np.diag(perr))
print(popt, perr)
G_gainfit = popt[0]

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

ifig = 0
for name in names:
    for idrift, tdrift in enumerate(driftbins[1:]):
        plt.figure(ifig)
        ifig += 1

        for ipitch, pitch in enumerate(pitchbins[:1]):
            toplot = (mpv_df.rr > 80) & mpv_df[name] & (mpv_df.pitch == pitch) & (mpv_df.tdrift == tdrift) & (mpv_df.muon)
            
            _ = plt.errorbar(mpv_df.rr[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                                  color=colors[ipitch], ls="none", fmt='o', capsize=3)
            plt.plot(mpv_df.rr[toplot], recombination_box(mpv_df.dedx_exp[toplot]) / G_gainfit, color=colors[ipitch])
        plt.ylabel("MPV dQ/dx [ADC/cm]")
        plt.xlabel("Residual Range [cm]")
        tlo = mpv_df[toplot].tlo.iloc[0]
        thi = mpv_df[toplot].thi.iloc[0]
        plt.text(0.5, 0.65, name.replace("_", " ") + " Muons\n$%.0f < t_\\mathrm{drift}$ < %.0f $\\mu$s\n0.3 < pitch < 0.4cm\n$70^\\circ < \\phi < 85^\\circ$" % (tlo, thi), 
                 fontsize=14, transform=plt.gca().transAxes)
        plt.tight_layout()
        if dosave:
            plt.savefig(savedir + "muon_RR_gainfit_dqdx_tdrift%i.pdf" % int(tlo))      
            plt.savefig(savedir + "muon_RR_gainfit_dqdx_tdrift%i.svg" % int(tlo))      

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

ifig = 0
for name in names:
    for idrift, tdrift in enumerate(driftbins[1:]):
        plt.figure(ifig)
        ifig += 1
        
        for ipitch, pitch in enumerate(pitchbins[:1]):
            toplot = (mpv_df.rr > 80) & mpv_df[name] & (mpv_df.pitch == pitch) & (mpv_df.tdrift == tdrift) & (mpv_df.muon)
            
            _ = plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                                  color="black", ls="none", fmt='o', capsize=3, label="Data")
            plt.plot(mpv_df.dedx_exp[toplot], recombination_box(mpv_df.dedx_exp[toplot]) / G_gainfit, label="Gain-Only Fit")

        plt.ylabel("MPV dQ/dx [ADC/cm]")
        plt.xlabel("Expected dE/dx [MeV/cm]")
        tlo = mpv_df[toplot].tlo.iloc[0]
        thi = mpv_df[toplot].thi.iloc[0]
        plt.text(0.5, 0.05, name.replace("_", " ") + " Muons\n$%.0f < t_\\mathrm{drift}$ < %.0f $\\mu$s\n0.3 < pitch < 0.4cm\n$70^\\circ < \\phi < 85^\\circ$\n$5^\\circ < \\theta_{xw} < 20^\\circ$" % (tlo, thi), 
                 fontsize=14, transform=plt.gca().transAxes)
        plt.legend(ncol=2, loc = "upper left")
        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "muon_dEdx_dqdx_gainfit_tdrift%i_%s.pdf" % (int(tlo), name))
            plt.savefig(savedir + "muon_dEdx_dqdx_gainfit_tdrift%i_%s.svg" % (int(tlo), name))

In [ ]:
fit_X = np.stack((mpv_df.dedx_exp[mpv_valid], mpv_df.phi[mpv_valid], mpv_df.muon[mpv_valid]))
fit_Y = mpv_df[mpv_valid].mpv
fit_Y_err =  mpv_df[mpv_valid].mpv_err.copy()
#fit_Y_err[mpv_df.muon] *= 10

In [ ]:
def fit_recombination(X, *param, MODBOX=True, ELLIPTICALBOX=False):
    if MODBOX:
        G = param[0]
        A = param[1]
        Bs = param[2:]
    else:
        G = param[0]
        if ELLIPTICALBOX:
            A = 1
        else:
            A = 0.8
        Bs = param[1:]
    
    dedx, phi, is_muon = X
    
    Bfit = np.zeros(dedx.shape)
    
    for i,p in enumerate(phis):
        Bfit[phi == p] = Bs[i]
        
    # For the muons, average the last two bins
    Bfit[is_muon==1] = (Bs[-1] + Bs[-2]) / 2
    
    recombination = None
    if MODBOX:
        recombination = recombination_box
    elif ELLIPTICALBOX:
        recombination = recombination_elbox
    else:
        recombination = recombination_birks
    
    return recombination(dedx, A, Bfit) / G
    

In [ ]:
fit_X = np.stack((mpv_df.dedx_exp[mpv_valid], mpv_df.phi[mpv_valid], mpv_df.muon[mpv_valid]))
fit_Y = mpv_df[mpv_valid].mpv
fit_Y_err =  mpv_df[mpv_valid].mpv_err.copy()

In [ ]:
# MODIFIED BOX FIT

In [ ]:

p0 = [
    80,
    0.93,
] + [0.212]*nphi

plo = [
    60,
    0.85,
] + [0.15]*nphi
phi = [
    90,
    1,
] + [0.35]*nphi


popt_box, perr_box = curve_fit(lambda x, *p: fit_recombination(x, *p, MODBOX=True), fit_X, fit_Y, p0=p0, bounds=(plo, phi),
                           sigma=fit_Y_err, absolute_sigma=True)

err_box = np.sqrt(np.diag(perr_box))
labels = ["Gain", "$\\alpha$"] + ["$\\beta(\\overline{%i})$" % int(phi) for phi in phis]

In [ ]:
for l, o, e in zip(labels, popt_box, err_box):
    if l.startswith("$\\beta"):
        e = np.sqrt(e**2 + (EFIELD_ERR*o)**2)
        
    print(l, o, e)

In [ ]:
0.2*0.017

In [ ]:
# BIRKS FIT

In [ ]:
p0 = [
    80,
] + [0.0486]*nphi

plo = [
    60,
] + [0.03]*nphi
phi = [
    90,
] + [0.06]*nphi

popt_birks, perr_birks = curve_fit(lambda x, *p: fit_recombination(x, *p, MODBOX=False), fit_X, fit_Y, p0=p0, bounds=(plo, phi),
                           sigma=fit_Y_err, absolute_sigma=True)

err_birks = np.sqrt(np.diag(perr_birks))

labels_birks = ["G"] + ["k %i" % int(phi) for phi in phis]

In [ ]:
for l, o, e in zip(labels_birks, popt_birks, err_birks):
    print(l, o, e)

In [ ]:
# Elliptical BOX FIT

In [ ]:
# p0 = [
#     80,
# ] + [0.3]*nphi

# plo = [
#     60,
# ] + [0.1]*nphi
# phi = [
#     90,
# ] + [0.7]*nphi

# popt_elbox, perr_elbox = curve_fit(lambda x, *p: fit_recombination(x, *p, MODBOX=False, ELLIPTICALBOX=True), fit_X, fit_Y, p0=p0, bounds=(plo, phi),
#                            sigma=fit_Y_err, absolute_sigma=True)

# err_elbox = np.sqrt(np.diag(perr_elbox))

# labels_elbox = ["G"] + ["x %i" % int(phi) for phi in phis]

In [ ]:
# for l, o, e in zip(labels_elbox, popt_elbox, err_elbox):
#     print(l, o, e)

In [ ]:
G = popt_box[0]
Gerr = err_box[0]
A = popt_box[1]
Aerr = err_box[1]
Bs = popt_box[2:]
Berr = err_box[2:]
Bmuon = (popt_box[-1] + popt_box[-2]) / 2


G_birks = popt_birks[0]
ks = popt_birks[1:]
kmuon = (popt_birks[-1] + popt_birks[-2]) / 2

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

ifig = 0
for name in names:
    for idrift, tdrift in enumerate(driftbins[1:]):
        plt.figure(ifig)
        ifig += 1

        for ipitch, pitch in enumerate(pitchbins[:1]):
            toplot = (mpv_df.rr > 80) & mpv_df[name] & (mpv_df.pitch == pitch) & (mpv_df.tdrift == tdrift) & (mpv_df.muon)
            
            _ = plt.errorbar(mpv_df.rr[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                                  color=colors[ipitch], ls="none", fmt='o', capsize=3)
            plt.plot(mpv_df.rr[toplot], recombination_box(mpv_df.dedx_exp[toplot], A, Bmuon) / G, color=colors[ipitch])
        plt.ylabel("MPV dQ/dx [ADC/cm]")
        plt.xlabel("Residual Range [cm]")
        tlo = mpv_df[toplot].tlo.iloc[0]
        thi = mpv_df[toplot].thi.iloc[0]
        plt.text(0.5, 0.65, name.replace("_", " ") + " Muons\n$%.0f < t_\\mathrm{drift}$ < %.0f $\\mu$s\n0.3 < pitch < 0.4cm\n$70^\\circ < \\phi < 85^\\circ$" % (tlo, thi), 
                 fontsize=14, transform=plt.gca().transAxes)
        plt.tight_layout()
        if dosave:
            plt.savefig(savedir + "muon_RR_dqdx_tdrift%i.pdf" % int(tlo))
            plt.savefig(savedir + "muon_RR_dqdx_tdrift%i.svg" % int(tlo))
        
        

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

ifig = 0
for name in names:
    for idrift, tdrift in enumerate(driftbins[1:]):
        plt.figure(ifig)
        ifig += 1
        
        for ipitch, pitch in enumerate(pitchbins[:1]):
            toplot = (mpv_df.rr > 80) & mpv_df[name] & (mpv_df.pitch == pitch) & (mpv_df.tdrift == tdrift) & (mpv_df.muon)
            
            _ = plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                                  color="black", ls="none", fmt='o', capsize=3, label="Data")
            plt.plot(mpv_df.dedx_exp[toplot], recombination_box(mpv_df.dedx_exp[toplot], A, Bmuon) / G, label="Mod. Box Fit")
            plt.plot(mpv_df.dedx_exp[toplot], recombination_birks(mpv_df.dedx_exp[toplot], B=kmuon) / G_birks, label="Birks Fit")
#             plt.plot(mpv_df.dedx_exp[toplot], recombination_box(mpv_df.dedx_exp[toplot]) / G_gainfit, label="Gain-Only Fit")

        plt.ylabel("MPV dQ/dx [ADC/cm]")
        plt.xlabel("Expected dE/dx [MeV/cm]")
        tlo = mpv_df[toplot].tlo.iloc[0]
        thi = mpv_df[toplot].thi.iloc[0]
        plt.text(0.5, 0.05, name.replace("_", " ") + " Muons\n$%.0f < t_\\mathrm{drift}$ < %.0f $\\mu$s\n0.3 < pitch < 0.4cm\n$70^\\circ < \\phi < 85^\\circ$\n$5^\\circ < \\theta_{xw} < 20^\\circ$" % (tlo, thi), 
                 fontsize=14, transform=plt.gca().transAxes)
        plt.legend(ncol=2, loc = "upper left")
        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "muon_dEdx_dqdx_tdrift%i_%s.pdf" % (int(tlo), name))
            plt.savefig(savedir + "muon_dEdx_dqdx_tdrift%i_%s.svg" % (int(tlo), name))

In [ ]:
for i, phi in enumerate(phis):
    plt.figure(i)
    toplot = ~mpv_df.muon & (mpv_df.phi == phi)
    philo = mpv_df[toplot].philo.iloc[0]
    phihi = mpv_df[toplot].phihi.iloc[0]
    _ = plt.errorbar(mpv_df.rr[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                          color="black", ls="none", fmt='o', capsize=3, label="Data")
    plt.plot(mpv_df.rr[toplot], recombination_box(mpv_df.dedx_exp[toplot], A, Bs[i]) / G, label="Mod. Box Fit")
    plt.plot(mpv_df.rr[toplot], recombination_birks(mpv_df.dedx_exp[toplot],B=ks[i]) / G_birks, label="Birks Fit")
#     plt.plot(mpv_df.rr[toplot], recombination_box(mpv_df.dedx_exp[toplot]) / G_gainfit, label="Gain-Only Fit")

    
    plt.ylabel("MPV dQ/dx [ADC/cm]")
    plt.xlabel("Residual Range [cm]")
    plt.text(0.6, 0.45, "Protons\n$%i^\\circ < \\phi < %i^\\circ$" % (philo, phihi), 
             fontsize=14, transform=plt.gca().transAxes)  
    plt.tight_layout()
    plt.legend()

    
    if dosave:
        plt.savefig(savedir + "proton_RR_dqdx_phi%i.pdf" % int(phi))
        plt.savefig(savedir + "proton_RR_dqdx_phi%i.svg" % int(phi))

In [ ]:
for i, phi in enumerate(phis):
    plt.figure(i)
    toplot = ~mpv_df.muon & (mpv_df.phi == phi)
    philo = mpv_df[toplot].philo.iloc[0]
    phihi = mpv_df[toplot].phihi.iloc[0]
    _ = plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                          color="black", ls="none", fmt='o', capsize=3, label="Data")
    
    modbox_v = recombination_box(mpv_df.dedx_exp[toplot], A, Bs[i]) / G
    chi2_modbox = np.sum(((mpv_df.mpv[toplot] - modbox_v) / mpv_df.mpv_err[toplot])**2)
    plt.plot(mpv_df.dedx_exp[toplot], modbox_v, 
#              label="Mod. Box Fit\n$\\chi^2/n = %.1f/%i$" % (chi2_modbox, len(modbox_v))
             label="Modified Box Fit"
            )
    
    birks_v = recombination_birks(mpv_df.dedx_exp[toplot],B=ks[i]) / G_birks
    chi2_birks = np.sum(((mpv_df.mpv[toplot] - birks_v) / mpv_df.mpv_err[toplot])**2)
    plt.plot(mpv_df.dedx_exp[toplot], birks_v, 
#              label="Birks Fit\n$\\chi^2/n = %.1f/%i$" % (chi2_birks, len(birks_v))
             label="Birks Fit"
            )
    
    print("$%.0f^\\circ < \\phi < %.0f^\\circ$ & $%.1f/%i$ & $%.1f/%i$\\\\" % (philo, phihi, chi2_birks, len(mpv_df.mpv[toplot]), chi2_modbox, len(mpv_df.mpv[toplot])))
    
#     gain_v = recombination_box(mpv_df.dedx_exp[toplot]) / G_gainfit
#     chi2_gain = np.sum(((mpv_df.mpv[toplot] - gain_v) / mpv_df.mpv_err[toplot])**2)
#     plt.plot(mpv_df.dedx_exp[toplot], gain_v, 
#              label="Gain-Only Fit\n$\\chi^2/n = %.1f/%i$" % (chi2_gain, len(gain_v)))

#   Plot Elliptical box
#     elbox_v = recombination_elbox(mpv_df.dedx_exp[toplot], 1, popt_elbox[i+1])/popt_elbox[0]
#     plt.plot(mpv_df.dedx_exp[toplot], elbox_v, label="El. Box Fit")

    plt.ylabel("Proton MPV dQ/dx [ADC/cm]")
    plt.xlabel("Proton Expected dE/dx [MeV/cm]")
#     plt.text(0.05, 0.6, "Protons\n$%i^\\circ < \\phi < %i^\\circ$\n0.3 < pitch < 1 cm\n$5^\\circ < \\theta_{xw} < 70^\\circ$" % (philo, phihi), 
#              fontsize=14, transform=plt.gca().transAxes)  
    plt.text(0.3, 0.95, "$%i^\\circ < \\phi < %i^\\circ$" % (philo, phihi), 
             fontsize=20, transform=plt.gca().transAxes, verticalalignment="top")  
    plt.legend(loc="lower right", fontsize=13)
    plt.tight_layout()

    if dosave:
        plt.savefig(savedir + "proton_RR_dEdx_phi%i.pdf" % int(phi))
        plt.savefig(savedir + "proton_RR_dEdx_phi%i.svg" % int(phi))
    

In [ ]:
mpv_df.philo

In [ ]:
cmap = mpl.cm.get_cmap('plasma')

colors = [cmap(c) for c in np.linspace(0.2, 1, len(phis))]

In [ ]:
toplot = mpv_df.muon & (mpv_df.rr > 80)
plt.plot(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot]*G/1e3,
                      color="black", ls="none", marker=".",
                 label="Muon MIPs")
dedxs = np.linspace(1.6, 12.5, 101)
for i, phi in enumerate((phis)):
    toplot = ~mpv_df.muon & (mpv_df.phi == phi)
    philo = mpv_df[toplot].philo.iloc[0]
    phihi = mpv_df[toplot].phihi.iloc[0]
    _ = plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot]*G/1e3, mpv_df.mpv_err[toplot]*G/1e3,
                          color=colors[i], ls="none", fmt='o', capsize=3, 
                     label="Proton $%i^\\circ < \\phi < %i^\\circ$" % (philo, phihi))
    plt.plot(dedxs, recombination_box(dedxs, A, Bs[i])/1e3, 
             color=colors[i])

plt.legend(ncol=2, loc='upper center', bbox_to_anchor=(0.5, 1.425))
plt.ylabel("Measured dQ/dx [ke$^-$/cm]")
plt.xlabel("Expected dE/dx [MeV/cm]")
# plt.tight_layout()

lc = mc.LineCollection([[(8, 50), (9, 50)]], colors="gray", linewidths=2)
plt.gca().add_collection(lc)
plt.text(9.25, 47, "Mod. Box Fits", fontsize=14)

if dosave:
    plt.savefig(savedir + "allparticles_dEdx.pdf", bbox_inches="tight")
    plt.savefig(savedir + "allparticles_dEdx.svg", bbox_inches="tight")
    

In [ ]:
toplot = mpv_df.muon & (mpv_df.rr > 80)

dedxs = np.linspace(4.5, 12.5, 101)
for i, phi in enumerate((phis)):
    toplot = ~mpv_df.muon & (mpv_df.phi == phi)
    philo = mpv_df[toplot].philo.iloc[0]
    phihi = mpv_df[toplot].phihi.iloc[0]
    _ = plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot]*G/1e3, mpv_df.mpv_err[toplot]*G/1e3,
                          color=colors[i], ls="none", fmt='o', capsize=3, 
                     label="$%i^\\circ < \\phi < %i^\\circ$" % (philo, phihi))
#     _ = plt.plot(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot]*G/1e3,
#                           color=colors[i], ls="none", marker='.', markersize=8,
#                      label="$%i^\\circ < \\phi < %i^\\circ$" % (philo, phihi))
    plt.plot(dedxs, recombination_box(dedxs, A, Bs[i])/1e3, 
             color=colors[i])

plt.legend(ncol=3, loc='upper center', bbox_to_anchor=(0.5, 1.325), title="Protons", 
           columnspacing=0.5, handletextpad=0)
plt.ylabel("Measured dQ/dx [ke$^-$/cm]")
plt.xlabel("Expected dE/dx [MeV/cm]")
# plt.tight_layout()

lc = mc.LineCollection([[(4.5, 210), (5.125, 210)]], colors="gray", linewidths=2)
plt.gca().add_collection(lc)
plt.text(5.25, 208, "Mod. Box Fits", fontsize=14)

plt.ylim([plt.ylim()[0]*0.85, plt.ylim()[1]])
# print(plt.gca().bbox)
# Inset muon fits
axins = inset_axes(plt.gca(), width="40%", height="40%", loc="lower right", bbox_to_anchor=(0, 0.125, 1, 1),
                   bbox_transform=plt.gca().transAxes)

axins.plot(mpv_df.dedx_exp[whenmu], mpv_df.mpv[whenmu]*G/1e3,
                      color="black", ls="none", marker=".")

dedxs_toplot = np.linspace(1.625, 1.72, 100)
axins.plot(dedxs_toplot, recombination_box(dedxs_toplot, A, Bmuon)/1e3, color="gray", zorder=5)

axins.tick_params(labelsize=11)
axins.set_xlabel("dE/dx [MeV/cm]", fontsize=11)
axins.set_ylabel("dQ/dx [ke$^-$/cm]", fontsize=11)
axins.set_title("Muons $(70^\\circ<\\phi<85^\\circ)$", fontsize=11)

if dosave:
    plt.savefig(savedir + "allparticles_dEdx_v2.pdf", bbox_inches="tight")
    plt.savefig(savedir + "allparticles_dEdx_v2.svg", bbox_inches="tight")
    

In [ ]:
dedxs = np.linspace(1.6, 20, 101)
for i, phi in enumerate((phis)):
    toplot = ~mpv_df.muon & (mpv_df.phi == phi)
    philo = mpv_df[toplot].philo.iloc[0]
    phihi = mpv_df[toplot].phihi.iloc[0]
    plt.plot(dedxs, recombination_box(dedxs, A, Bs[i])/1e3, 
             color=colors[i], label="$%.0f^\\circ, %.3f$" % (phi, Bs[i]))

leg = plt.legend(ncol=2, loc="lower right", 
           title="$\\phi$, $\\beta(\\phi)$ [(kV/MeV)(g/mL)]",
          columnspacing=0.8, handletextpad=0.2, alignment="center")

plt.ylabel("dQ/dx [ke$^-$/cm]")
plt.xlabel("dE/dx [MeV/cm]")

plt.text(0.05, 0.95, "Modified Box Fits\n\n$\\alpha: %.3f$" % A, 
         fontsize=14, transform=plt.gca().transAxes, verticalalignment="top")

if dosave:
    plt.savefig(savedir + "allparticles_dEdx_fits.pdf", bbox_inches="tight")
    plt.savefig(savedir + "allparticles_dEdx_fits.svg", bbox_inches="tight")
    

In [ ]:
# Combined Muon plot

In [ ]:
if not isMC:
    cryos = [
        mpv_df.TPC_EE | mpv_df.TPC_EW,
        mpv_df.TPC_WE | mpv_df.TPC_WW,
    ]

    cryo_names = ["East", "West"]
    
else:
    cryos = [mpv_df.MC]
    cryo_names = ["MC"]

In [ ]:
for ifig, (c, cname) in enumerate(zip(cryos, cryo_names)):
    plt.figure(ifig)
    toplot = whenmu & c
    plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                                      color="black", ls="none", fmt='o', markersize=3, capsize=3)

    plt.ylabel("Muon MPV dQ/dx [ADC/cm]")
    plt.xlabel("Muon Expected dE/dx [MeV/cm]")

    dedxs_toplot = np.linspace(1.625, 1.72, 100)
    plt.plot(dedxs_toplot, recombination_box(dedxs_toplot, A, Bmuon) / G, label="Modified Box", zorder=5)
    plt.plot(dedxs_toplot, recombination_birks(dedxs_toplot, B=kmuon) / G_birks, label="Birks", zorder=5)

    plt.legend(title="Fit ($\\phi = 77.5^\\circ$)", loc="lower right")
    plt.text(0.675, 0.325, "%s Cryostat" % cname, fontsize=14, transform=plt.gca().transAxes)

    if dosave:
        plt.savefig(savedir + "allmuons_dEdx_Cryo%s.pdf" % cname, bbox_inches="tight")
        plt.savefig(savedir + "allmuons_dEdx_Cryo%s.svg" % cname, bbox_inches="tight")

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

ifig = 0
for name in names:
    for idrift, tdrift in enumerate(driftbins[1:]):
        plt.figure(ifig)
        ifig += 1
        
        for ipitch, pitch in enumerate(pitchbins[:1]):
            toplot = (mpv_df.rr > 80) & mpv_df[name] & (mpv_df.pitch == pitch) & (mpv_df.tdrift == tdrift) & (mpv_df.muon)
            
            _ = plt.errorbar(mpv_df.dedx_exp[toplot], mpv_df.mpv[toplot], mpv_df.mpv_err[toplot],
                                  color="black", ls="none", fmt='o', capsize=3, label="Data")
            plt.plot(mpv_df.dedx_exp[toplot], recombination_box(mpv_df.dedx_exp[toplot], A, Bmuon) / G, label="Mod. Box Fit")
            plt.plot(mpv_df.dedx_exp[toplot], recombination_birks(mpv_df.dedx_exp[toplot], B=kmuon) / G_birks, label="Birks Fit")

        plt.ylabel("MPV dQ/dx [ADC/cm]")
        plt.xlabel("Expected dE/dx [MeV/cm]")
        tlo = mpv_df[toplot].tlo.iloc[0]
        thi = mpv_df[toplot].thi.iloc[0]
        plt.text(0.5, 0.05, name.replace("_", " ") + " Muons\n$%.0f < t_\\mathrm{drift}$ < %.0f $\\mu$s\n0.3 < pitch < 0.4cm\n$70^\\circ < \\phi < 85^\\circ$\n$5^\\circ < \\theta_{xw} < 20^\\circ$" % (tlo, thi), 
                 fontsize=14, transform=plt.gca().transAxes)
        plt.legend(ncol=2, loc = "upper left")
        plt.tight_layout()

        if dosave:
            plt.savefig(savedir + "muon_dEdx_dqdx_tdrift%i_%s.pdf" % (int(tlo), name))
            plt.savefig(savedir + "muon_dEdx_dqdx_tdrift%i_%s.svg" % (int(tlo), name))

In [ ]:
 plt.figure(0, figsize=(20, 20))

plt_perr = np.copy(perr_box)
plt_perr[2:, 2:] += np.outer(popt_box[2:]*EFIELD_ERR, popt_box[2:]*EFIELD_ERR)
plt_err = np.sqrt(np.diag(plt_perr))

pltmatrix = plt_perr*np.outer(1/plt_err, 1/plt_err)
plt.matshow(pltmatrix, cmap="seismic", vmin=-1, vmax=1)

for i in range(plt_perr.shape[0]):
    for j in range(plt_perr.shape[1]):
        c = pltmatrix[j,i]
        plt.gca().text(i, j, "%.2f" % c, va='center', ha='center', color="white", fontsize=8)
        
plt.gca().set_xticklabels(['']+labels, fontsize=8)
_ = plt.gca().set_yticklabels(['']+labels, fontsize=8)

cbar = plt.colorbar()
for t in cbar.ax.get_yticklabels():
     t.set_fontsize(10)

if dosave:
    plt.savefig(savedir + "modbox_correlation.pdf", bbox_inches="tight")
    plt.savefig(savedir + "modbox_correlation.svg", bbox_inches="tight")

In [ ]:
plt.errorbar(phis, Bs, yerr=Berr, xerr=(phihis-philos)/2, linestyle="none", color="black", marker=".")
plt.xlabel("$\\phi$")
plt.ylabel("Mod. Box $\\beta$")
plt.tight_layout()

if isMC:
    plt.axhline([0.212])

    

if dosave:
    plt.savefig(savedir + "beta_measurement_perphi.pdf")
    plt.savefig(savedir + "beta_measurement_perphi.svg")

In [ ]:
Bfrac = Bs / Bs[-1]

Bind = 2

Bfrac_err = np.sqrt(np.maximum((Berr*Bfrac/Bs)**2 + (Berr[-1]*Bfrac/Bs[-1])**2 - 2*perr_box[Bind:,-1]*Bfrac/Bs/Bs[-1], 0))

plt.errorbar(phis, Bfrac, yerr=Bfrac_err, xerr=(phihis-philos)/2, linestyle="none", color="black", marker=".")
plt.xlabel("$\\phi$")
plt.ylabel("Mod. Box $\\beta(\\phi)$ / $\\beta(82.5^\\circ)$")
plt.tight_layout()

if dosave:
    plt.savefig(savedir + "beta_measurement_ratio_perphi.pdf")
    plt.savefig(savedir + "beta_measurement_ratio_perphi.svg")

In [ ]:
# fit functions
def fit_const(phi, c):
    return np.array([c]*len(phi))

def fit_lin(phi, m, b):
    return m*phi+b

def fit_quad(phi, a, b, c):
    return a*phi**2 + b*phi + c

def fit_columnar(phi, B0):
    return B0/np.sin(phi*np.pi/180)

def fit_ellipsoid(phi, B0, R):
    return B0/np.sqrt(np.sin(phi*np.pi/180)**2 + np.cos(phi*np.pi/180)**2/R**2)

In [ ]:
plt.errorbar(phis, Bs, yerr=Berr, xerr=(phihis-philos)/2, linestyle="none", color="black", marker=".", label="Data")
ylo, yhi = plt.ylim()

popt, _ = curve_fit(fit_columnar, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
plt.plot(phis, fit_columnar(phis, *popt), label="Columnar Fit")

popt, _ = curve_fit(fit_ellipsoid, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
plt.plot(phis, fit_ellipsoid(phis, *popt), label="Ellipsoid Fit")


popt, _ = curve_fit(fit_const, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
plt.plot(phis, fit_const(phis, *popt), label="Constant Fit")

popt, _ = curve_fit(fit_lin, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
plt.plot(phis, fit_lin(phis, *popt), label="Linear Fit")
print(popt)

popt, _ = curve_fit(fit_quad, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
plt.plot(phis, fit_quad(phis, *popt), label="Quadratic Fit")
print(popt)



# plt.plot(phis, [np.mean(Bs)]*len(phis), label="Constant Fit")
# plt.plot(phis, 0.18/np.sin(phis*np.pi/180), label="1/sin($\\alpha$) Fit")
# plt.plot(phis, 0.261 - 0.001*phis, label="Linear Fit")

plt.ylim([ylo, yhi])

plt.xlabel("$\\phi$")
plt.ylabel("Mod. Box $\\beta$")
plt.legend()
plt.tight_layout()
if dosave:
    plt.savefig(savedir + "beta_measurement_fits_perphi.pdf")
    plt.savefig(savedir + "beta_measurement_fits_perphi.svg")
    

In [ ]:
Bfrac = Bs / Bs[-1]

Bind = 2

Bfrac_err = np.sqrt(np.maximum((Berr*Bfrac/Bs)**2 + (Berr[-1]*Bfrac/Bs[-1])**2 - 2*perr_box[Bind:,-1]*Bfrac/Bs/Bs[-1], 0))

hdata = plt.errorbar(phis, Bfrac, yerr=Bfrac_err, xerr=(phihis-philos)/2, 
                     linestyle="none", color="black", marker=".", label="Data")

ylim = plt.ylim()

phiplot = np.linspace(phis[0], phis[-1], 500)

popt, _ = curve_fit(fit_columnar, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
hcol = plt.plot(phiplot, fit_columnar(phiplot, *popt) / fit_columnar(phiplot[-1:], *popt), label="Columnar")[0]

popt, _ = curve_fit(fit_const, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
hconst = plt.plot(phiplot, fit_const(phiplot, *popt) / fit_const(phiplot[-1:], *popt), label="Constant")[0]

popt, perr = curve_fit(fit_ellipsoid, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
helips = plt.plot(phiplot, fit_ellipsoid(phiplot, *popt) / fit_ellipsoid(phiplot[-1:], *popt), label="Ellipsoid")[0]

plt.ylim(ylim)

leg = plt.legend(handles=[hcol, hconst, helips], title="Angular\nDependence")

plt.legend(handles=[hdata], labels=["Data"], frameon=False, loc="center right")
plt.gca().add_artist(leg)


plt.xlabel("$\\phi$ [$^\\circ$]")
plt.ylabel("Mod. Box $\\beta(\\phi)$ / $\\beta(82.5^\\circ)$")
plt.tight_layout()

if dosave:
    plt.savefig(savedir + "beta_measurement_ratio_wfits_perphi.pdf")
    plt.savefig(savedir + "beta_measurement_ratio_wfits_perphi.svg")

In [ ]:
Bfrac = Bs / Bs[-1]

Bind = 2

Bfrac_err = np.sqrt(np.maximum((Berr*Bfrac/Bs)**2 + (Berr[-1]*Bfrac/Bs[-1])**2 - 2*perr_box[Bind:,-1]*Bfrac/Bs/Bs[-1], 0))

hdata = plt.errorbar(phis, Bfrac, yerr=Bfrac_err, xerr=(phihis-philos)/2, 
                     linestyle="none", color="black", marker=".", label="Data")

ylim = plt.ylim()

phiplot = np.linspace(phis[0], phis[-1], 500)

popt, _ = curve_fit(fit_columnar, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
hcol = plt.plot(phiplot, fit_columnar(phiplot, *popt) / fit_columnar(phiplot[-1:], *popt), label="Columnar")[0]

popt, _ = curve_fit(fit_const, phis, Bs,
                           sigma=Berr, absolute_sigma=True)
hconst = plt.plot(phiplot, fit_const(phiplot, *popt) / fit_const(phiplot[-1:], *popt), label="Constant")[0]

plt.ylim(ylim)

leg = plt.legend(handles=[hcol, hconst], title="Angular\nDependence")

plt.legend(handles=[hdata], labels=["Data"], frameon=False, loc="center right")
plt.gca().add_artist(leg)


plt.xlabel("$\\phi$ [$^\\circ$]")
plt.ylabel("Mod. Box $\\beta(\\phi)$ / $\\beta(82.5^\\circ)$")
plt.tight_layout()

if dosave:
    plt.savefig(savedir + "beta_measurement_ratio_wfits_perphi_noellipse.pdf")
    plt.savefig(savedir + "beta_measurement_ratio_wfits_perphi_noellipse.svg")

In [ ]:
# BETA
plt.errorbar(phis, Bs, yerr=np.sqrt(Berr**2 + (EFIELD_ERR*Bs)**2), xerr=(phihis-philos)/2, 
             linestyle="none", color="black", marker=".", label="ICARUS")

argoneut_angles = np.array([20, 47, 55, 70, 90] )
argoneut_angle_los = argoneut_angles[:-1]
argoneut_angle_his = argoneut_angles[1:]
argoneut_angle_vals = np.array([40, 50, 60, 80])

argoneut_angle_difflo = argoneut_angle_vals - argoneut_angle_los
argoneut_angle_diffhi = argoneut_angle_his - argoneut_angle_vals

argoneut_vals = np.array([0.346, 0.327, 0.317, 0.302])/(0.319/0.212)
argoneut_errs = np.array([0.007, 0.007, 0.006, 0.005]) / (0.319/0.212)

plt.errorbar(argoneut_angle_vals, argoneut_vals, 
             yerr=argoneut_errs, 
             xerr=(argoneut_angle_difflo, argoneut_angle_diffhi), marker=".",
             linestyle="none", color="green", label="ArgoNeuT")

#plt.errorbar((20+90)/2, 0.184, yerr=0.002, xerr=(90-20)/2, linestyle="none", marker=".", color="blue", label="MicroBooNE")
#plt.errorbar(80, 0.194, yerr=0.005, xerr=10, linestyle="none", marker=".", color="brown", label="ProtoDUNE")

plt.xlabel("$\\phi$ [$^\\circ$]")
plt.ylabel("Mod. Box $\\beta$ [(kV/MeV)(g/mL)]")
#plt.legend(ncol=2, loc='upper center', bbox_to_anchor=(0.5, 1.3))
plt.legend()
plt.tight_layout()
if dosave:
    plt.savefig(savedir + "beta_measurement_argoneut_perphi.pdf")
    plt.savefig(savedir + "beta_measurement_argoneut_perphi.svg")

In [ ]:
# ALPHA
plt.errorbar((phis[-1] + phis[0]) / 2, A, yerr=Aerr, xerr=(phis[-1] - phis[0])/2,
            linestyle="none", color="black", marker=".", label="ICARUS")
argoneut_a_vals = [1.06, 0.90, 0.92, 0.91]
argoneut_a_errs = [0.05, 0.04, 0.04, 0.03]

plt.errorbar(argoneut_angle_vals, argoneut_a_vals, 
             yerr=argoneut_a_errs, 
             xerr=(argoneut_angle_difflo, argoneut_angle_diffhi), marker=".",
             linestyle="none", color="green", label="ArgoNeuT")

plt.ylabel("Mod. Box $\\alpha$")
plt.xlabel("$\\phi$ [$^\\circ$]")

#plt.legend(ncol=2, loc='upper center', bbox_to_anchor=(0.5, 1.3))
plt.legend()
plt.tight_layout()
if dosave:
    plt.savefig(savedir + "alpha_measurement_argoneut_perphi.pdf")
    plt.savefig(savedir + "alpha_measurement_argoneut_perphi.svg")

In [ ]:
print(popt_box)

In [ ]:
# Ellipsoid Modified Box (EMB) fit

In [ ]:
1+1

In [ ]:
def fit_emb_recombination(X, G, A, beta0, R):
    dedx, phi, is_muon = X
    Bs = beta0 / np.sqrt(np.sin(phi*np.pi/180)**2 + np.cos(phi*np.pi/180)**2/R**2)
    
    Bfit = np.zeros(dedx.shape)
    
    for i,p in enumerate(phi):
        Bfit[phi == p] = Bs[i]
        
    # For the muons, average the last two bins
    Bfit[is_muon==1] = (Bs[-1] + Bs[-2]) / 2
    
    return recombination_box(dedx, A, Bfit) / G
    

In [ ]:
p0 = [
    80,
    0.93,
    0.2,
    1.25
]

plo = [
    60,
    0.85,
    0.1,
    0.1,
]
phi = [
    90,
    1,
    0.3,
    100
]


popt_emb, perr_emb = curve_fit(lambda x, *p: fit_emb_recombination(x, *p), fit_X, fit_Y, p0=p0, bounds=(plo, phi),
                           sigma=fit_Y_err, absolute_sigma=True)

err_emb = np.sqrt(np.diag(perr_emb))
labels = ["$\\mathcal{G}$", "$\\alpha$", "$\\beta_{90}$", "R"]

In [ ]:
for l, o, e in zip(labels, popt_emb, err_emb):
    if l.startswith("$\\beta"):
        e = np.sqrt(e**2 + (EFIELD_ERR*o)**2)
        
    print(l, o, e)

In [ ]:
 plt.figure(0, figsize=(20, 20))

plt_perr = np.copy(perr_emb)
plt_perr[2, 2] += (popt_emb[2]*EFIELD_ERR)**2
plt_err = np.sqrt(np.diag(plt_perr))

pltmatrix = plt_perr*np.outer(1/plt_err, 1/plt_err)
plt.matshow(pltmatrix, cmap="seismic", vmin=-1, vmax=1)

for i in range(plt_perr.shape[0]):
    for j in range(plt_perr.shape[1]):
        c = pltmatrix[j,i]
        plt.gca().text(i, j, "%.2f" % c, va='center', ha='center', color="white", fontsize=8)
        
plt.gca().set_xticklabels(['']+labels, fontsize=14)
_ = plt.gca().set_yticklabels(['']+labels, fontsize=14)

cbar = plt.colorbar()
for t in cbar.ax.get_yticklabels():
     t.set_fontsize(10)
cbar.set_label("Correlation", rotation=270, labelpad=13)

if dosave:
    plt.savefig(savedir + "emb_correlation.pdf", bbox_inches="tight")
    plt.savefig(savedir + "emb_correlation.svg", bbox_inches="tight")

In [ ]:
# Try fit w/out muons
nomu_fit_X = np.stack((mpv_df.dedx_exp[mpv_valid & ~mpv_df.muon], mpv_df.phi[mpv_valid & ~mpv_df.muon], mpv_df.muon[mpv_valid & ~mpv_df.muon]))
nomu_fit_Y = mpv_df[mpv_valid & ~mpv_df.muon].mpv
nomu_fit_Y_err =  mpv_df[mpv_valid & ~mpv_df.muon].mpv_err.copy()

p0 = [
    80,
    0.95,
    0.2,
    1.25
]

plo = [
    60,
    0.80,
    0.1,
    0.1,
]
phi = [
    90,
    1,
    0.3,
    100
]


popt_nomu_emb, perr_nomu_emb = curve_fit(lambda x, *p: fit_emb_recombination(x, *p), nomu_fit_X, nomu_fit_Y, p0=p0, bounds=(plo, phi),
                           sigma=nomu_fit_Y_err, absolute_sigma=True)

err_nomu_emb = np.sqrt(np.diag(perr_nomu_emb))

In [ ]:
for l, o, e in zip(labels, popt_nomu_emb, err_nomu_emb):
    if l.startswith("$\\beta"):
        e = np.sqrt(e**2 + (EFIELD_ERR*o)**2)
        
    print(l, o, e)

In [ ]:
# Try fit w/out diffusion in muon dE/dx
nomu_fit_X = np.stack((mpv_df.dedx_exp_pitch[mpv_valid], mpv_df.phi[mpv_valid], mpv_df.muon[mpv_valid]))
nomu_fit_Y = mpv_df[mpv_valid].mpv
nomu_fit_Y_err =  mpv_df[mpv_valid].mpv_err.copy()

p0 = [
    80,
    0.95,
    0.2,
    1.25
]

plo = [
    60,
    0.80,
    0.1,
    0.1,
]
phi = [
    90,
    1,
    0.3,
    100
]


popt_nodiffusion_emb, perr_nodiffusion_emb = curve_fit(lambda x, *p: fit_emb_recombination(x, *p), nomu_fit_X, nomu_fit_Y, p0=p0, bounds=(plo, phi),
                           sigma=nomu_fit_Y_err, absolute_sigma=True)

err_nodiffusion_emb = np.sqrt(np.diag(perr_nodiffusion_emb))

In [ ]:
for l, o, e in zip(labels, popt_nodiffusion_emb, err_nodiffusion_emb):
    if l.startswith("$\\beta"):
        e = np.sqrt(e**2 + (EFIELD_ERR*o)**2)
        
    print(l, o, e)

In [ ]:
def log_prob_emb_fit(fitvars):
    dedx, phi, is_muon = fit_X
    G, A, beta90, R = fitvars.T
    Bs = beta90 / np.sqrt(np.sin(phi*np.pi/180)**2 + np.outer(np.cos(phi*np.pi/180)**2, 1/R**2).T).T
    
    Bfit = np.zeros(Bs.shape)
    for i,p in enumerate(phi):
        Bfit[phi == p, :] = Bs[i]
        
    # For the muons, average the last two bins
    Bfit[is_muon==1, :] = (Bs[-1] + Bs[-2]) / 2
    Amat = np.broadcast_to(A, (dedx.size, A.size))
    Gmat = np.broadcast_to(G, (dedx.size, G.size))
    
    pred_y = np.nan_to_num(recombination_box(dedx, Amat.T, Bfit.T) / Gmat.T)
    delta = pred_y - fit_Y.values
    chi2 = np.sum(((pred_y - fit_Y.values)/fit_Y_err.values)**2, axis=-1)

#     chi2[G < 75] = np.inf
#     chi2[G > 75.2] = np.inf

    # remove unphysical regions of parameter space
    chi2[R < 1] = np.inf
    chi2[R > 5] = np.inf
    chi2[beta90 < 0] = np.inf
    
    return -chi2/2

In [ ]:
np.random.seed(42)

Ginit = np.random.normal(80, 5, size=32)
Ainit = np.random.normal(0.9, 0.05, size=32)
Binit = np.random.normal(0.2, 0.05, size=32)
Rinit = np.random.normal(5, 0.5, size=32)

initial = np.stack((Ginit, Ainit, Binit, Rinit)).T
initial.shape

In [ ]:
nwalkers, ndim = initial.shape
n_burn_in = 1000

n_sample = 10_000

with Pool(processes=24) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob_emb_fit, pool=pool)
    state = sampler.run_mcmc(initial, n_burn_in, progress=True)
    _ = sampler.run_mcmc(state, n_sample, progress=True)

In [ ]:
samples = sampler.get_chain(flat=True, discard=n_burn_in, thin=10)

In [ ]:
Gsample = samples[:, 0]
Asample = samples[:, 1]
Bsample = samples[:, 2]
Rsample = samples[:, 3]

In [ ]:
Rcut = 1.35

In [ ]:
_ = corner.corner(samples[Rsample < Rcut, :], labels=["$\\mathcal{G}$ [e$^-$/ADC]", "$\\alpha$", "$\\beta_{90}$ [(kV/MeV)(g/mL)]", "R"])

In [ ]:
def fit_gaus(X, A, mu, sigma):
    return A*np.exp(-(X-mu)**2/(2*sigma**2))

In [ ]:
N,bins,_ = plt.hist(Gsample[Rsample < Rcut], bins=20)
centers = (bins[1:] + bins[:-1])/2

p0 = [np.max(N), np.mean(centers), bins[-1] - bins[0]]

pfit, perr = curve_fit(fit_gaus, centers, N, p0=p0, sigma=np.maximum(np.sqrt(N), 1))
bplot = np.linspace(bins[0], bins[-1], 1001)

plt.plot(bplot, fit_gaus(bplot, *pfit))

print("Value: %.1f +/- %.1f" % (pfit[1], pfit[2]))

In [ ]:
N,bins,_ = plt.hist(Asample[Rsample < Rcut], bins=20)
centers = (bins[1:] + bins[:-1])/2

p0 = [np.max(N), np.mean(centers), bins[-1] - bins[0]]

pfit, perr = curve_fit(fit_gaus, centers, N, p0=p0, sigma=np.maximum(np.sqrt(N), 1))
bplot = np.linspace(bins[0], bins[-1], 1001)

plt.plot(bplot, fit_gaus(bplot, *pfit))

print("Value: %.3f +/- %.3f" % (pfit[1], pfit[2]))

In [ ]:
N,bins,_ = plt.hist(Bsample[Rsample < Rcut], bins=20)
centers = (bins[1:] + bins[:-1])/2

p0 = [np.max(N), np.mean(centers), bins[-1] - bins[0]]

pfit, perr = curve_fit(fit_gaus, centers, N, p0=p0, sigma=np.maximum(np.sqrt(N), 1))
bplot = np.linspace(bins[0], bins[-1], 1001)

plt.plot(bplot, fit_gaus(bplot, *pfit))

print("Value: %.3f +/- %.3f" % (pfit[1], pfit[2]))

In [ ]:
N,bins,_ = plt.hist(Rsample[Rsample < Rcut], bins=20)
centers = (bins[1:] + bins[:-1])/2

p0 = [np.max(N), np.mean(centers), bins[-1] - bins[0]]

pfit, perr = curve_fit(fit_gaus, centers, N, p0=p0, sigma=np.maximum(np.sqrt(N), 1))
bplot = np.linspace(bins[0], bins[-1], 1001)

plt.plot(bplot, fit_gaus(bplot, *pfit))

print("Value: %.4f +/- %.4f" % (pfit[1], pfit[2]))

In [ ]:
def emb_recombination_cor(dQdx, phi, A, B90, R, G):
    Bs = B90 / np.sqrt(np.moveaxis(np.sin(phi*np.pi/180)**2 + np.multiply.outer(1/R**2, np.cos(phi*np.pi/180)**2), 0, -1))
    
    dQdx_G = np.multiply.outer(dQdx, G)    
    return recombination_cor(dQdx_G, A, Bs)                   

In [ ]:
dQdx_lo = 500
dQdx_hi = 8500

phi_lo = 0
phi_hi = 90

ndQdx = 801
nphi = 19

In [ ]:
phis = np.tile(np.linspace(phi_lo, phi_hi, nphi), (ndQdx, 1)).T
dQdxs = np.tile(np.linspace(dQdx_lo, dQdx_hi, ndQdx), (nphi, 1))

In [ ]:
sample_ok = Rsample < Rcut

dEdxs = emb_recombination_cor(dQdxs, phis, Asample[sample_ok], Bsample[sample_ok], Rsample[sample_ok], Gsample[sample_ok])

In [ ]:
dEdx_mean = np.mean(dEdxs, -1)
dEdx_std = np.std(dEdxs, -1)

In [ ]:
# plt.figure()
fig, ax = plt.subplots(figsize=(6.4,4.8))

m = ax.matshow(dEdx_mean, origin="lower", 
            extent=[dQdx_lo, dQdx_hi, phi_lo, phi_hi], 
            aspect=(dQdx_hi-dQdx_lo)/(phi_hi-phi_lo),
           interpolation='none')

cbar = fig.colorbar(m)
plt.gca().xaxis.tick_bottom()

# cbar.set_label("dE/dx [MeV/cm]", rotation=270, labelpad=15)
plt.ylabel("Track Angle to Drift Field ($\\phi$) [$^\\circ$]")
plt.xlabel("$dQ/dx$ [ADC/cm]")
plt.title("Mean Calibrated dE/dx [MeV/cm]")

In [ ]:
# plt.figure()
fig, ax = plt.subplots(figsize=(6.4,4.8))

m = ax.matshow(dEdx_std*100/dEdx_mean, origin="lower", 
            extent=[dQdx_lo, dQdx_hi, phi_lo, phi_hi], 
            aspect=(dQdx_hi-dQdx_lo)/(phi_hi-phi_lo),
           interpolation='none')

cbar = fig.colorbar(m)
plt.gca().xaxis.tick_bottom()

# cbar.set_label("dE/dx [MeV/cm]", rotation=270, labelpad=15)
plt.ylabel("Track Angle to Drift Field ($\\phi$) [$^\\circ$]")
plt.xlabel("$dQ/dx$ [ADC/cm]")
plt.title("Uncertainty in Calibrated dE/dx [%]")

In [ ]:
# We want to express the uncertainty in terms of dE/dx, not dQ/dx

In [ ]:
dEdx_lo = 0.3
dEdx_hi = 150
ndEdx = 1498

In [ ]:
for i in range(len(phis.T[0])):    
    print(dEdx_mean[i,:].max())

In [ ]:
dEdxs = np.linspace(dEdx_lo, dEdx_hi, ndEdx)
dEdx_unc = np.zeros((len(phis.T[0]), ndEdx))

for i in range(len(phis.T[0])):    
    unc = dEdx_std[i, :]*100/dEdx_mean[i, :]
    dEdx_2_dEdx_unc = interp1d(dEdx_mean[i, :], unc, bounds_error=False, fill_value=(unc[0], unc[-1]))
    
    dEdx_unc[i, :] = dEdx_2_dEdx_unc(dEdxs)

In [ ]:
# plt.figure()
fig, ax = plt.subplots(figsize=(6.4,4.8))

dEdx_plt = 197

m = ax.matshow(dEdx_unc[:, :dEdx_plt], origin="lower", 
            extent=[dEdx_lo, dEdxs[dEdx_plt], phi_lo, phi_hi], 
            aspect=(dEdxs[dEdx_plt]-dEdx_lo)/(phi_hi-phi_lo),
           interpolation='none')

cbar = fig.colorbar(m)
plt.gca().xaxis.tick_bottom()

# cbar.set_label("dE/dx [MeV/cm]", rotation=270, labelpad=15)
plt.ylabel("Track Angle to Drift Field ($\\phi$) [$^\\circ$]")
plt.xlabel("$dE/dx$ [MeV/cm]")
plt.title("Uncertainty in Calibrated dE/dx [%]")

In [ ]:
if savedata:
    with open(savedir + "dedx_uncertainty.txt", "w+") as f:
        #f.write("\t")
        for p in phis.T[0]:
            f.write("\t%i" % p)
        f.write("\n")
        for i, dEdx in enumerate(dEdxs):
            f.write("%.1f" % dEdx)
            for j in range(len(phis.T[0])):
                v = dEdx_unc[j, i]
                f.write("\t%.4f" % v)
            f.write("\n")

In [ ]:
# Do the same for stubs: now we need to account for the bias in beta across the range of phi

In [ ]:
Amean = np.mean(Asample[sample_ok])
Bmean = np.mean(Bsample[sample_ok])
Rmean = np.mean(Rsample[sample_ok])
Gmean = np.mean(Gsample[sample_ok])

phi = 0
dEdx_phi0 = recombination_cor(dQdxs*Gmean, Amean, Bmean/np.sqrt(np.sin(phi)**2 + np.cos(phi)**2/Rmean**2))
phi = 90*np.pi/180
dEdx_phi90 = recombination_cor(dQdxs*Gmean, Amean, Bmean/np.sqrt(np.sin(phi)**2 + np.cos(phi)**2/Rmean**2))

dedx_phi_diff = np.abs(dEdx_phi0 - dEdx_phi90)/2

phiind = 0 # use phi=0
unc_phi = dedx_phi_diff[phiind, :]*100/dEdx_mean[phiind, :]
dEdx_2_dEdx_phi_unc = interp1d(dEdx_mean[phiind, :],
                               unc_phi, bounds_error=False, fill_value=(unc[0], unc[-1]))
    
dEdx_unc_wphi = np.sqrt(dEdx_unc**2 + dEdx_2_dEdx_phi_unc(dEdxs)**2)

In [ ]:
# plt.figure()
fig, ax = plt.subplots(figsize=(6.4,4.8))

dEdx_plt = 197

m = ax.matshow(dEdx_unc_wphi[:, :dEdx_plt], origin="lower", 
            extent=[dEdx_lo, dEdxs[dEdx_plt], phi_lo, phi_hi], 
            aspect=(dEdxs[dEdx_plt]-dEdx_lo)/(phi_hi-phi_lo),
           interpolation='none')

cbar = fig.colorbar(m)
plt.gca().xaxis.tick_bottom()

# cbar.set_label("dE/dx [MeV/cm]", rotation=270, labelpad=15)
plt.ylabel("Track Angle to Drift Field ($\\phi$) [$^\\circ$]")
plt.xlabel("$dE/dx$ [MeV/cm]")
plt.title("Uncertainty in Calibrated dE/dx (No $\\phi$ Corr.) [%]")

In [ ]:
if savedata:
    with open(savedir + "dedx_uncertainty_nophicorr.txt", "w+") as f:
        #f.write("\t")
        for p in phis.T[0]:
            f.write("\t%i" % p)
        f.write("\n")
        for i, dEdx in enumerate(dEdxs):
            f.write("%.1f" % dEdx)
            for j in range(len(phis.T[0])):
                v = dEdx_unc_wphi[j, i]
                f.write("\t%.4f" % v)
            f.write("\n")

In [ ]:
if savedata:
    with open(savedir + "popt_box.txt", "w+") as f:
        for p, e in zip(popt_box, err_box):
            f.write("%f\t%f\n" % (p, e))